# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 불가능


In [3]:
!nvidia-smi

Sat Sep 18 18:01:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 452.56       Driver Version: 452.56       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce MX450      WDDM  | 00000000:2D:00.0 Off |                  N/A |
| N/A   52C    P8    N/A /  N/A |    119MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2019년 Hyundai Data**

In [5]:
lexicon_2019 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2019_hyundai.csv')

lexicon_2019.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,현대차,5380,매일경제,2019010104,[국내 업종별 전망 / 자동차] 내수도 수출도 암울…6대 생산국 지위 흔들,http://news.mk.co.kr/newsRead.php?year=2019&no...,◆ 2019 경제기상도 ◆ \r\n\n\n 지난해 국내 자동차 업계는 한국GM의 군...,2019-01-02,4,117500,118000,113500,114000,565493,-0.037975,-1,-1,경제 기상도 지난해 국내 자동차 업계 한국 군산 공장 철수 시작 가쁜 한국 사태 지...


In [6]:
lexicon_2019[lexicon_2019['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization


In [7]:
lexicon_2019.dropna(axis=0, inplace=True)
lexicon_2019 = lexicon_2019.reset_index(drop=True)
lexicon_2019.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [10]:
pos_lexicon_2019 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_pos_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_pos와의 Cosine Similarity 계산
for x in range(len(lexicon_2019['Tokenization'])):
    news_num = x+1
    for y in range(len(positive)):
        for z in range(len(list(set(lexicon_2019['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(positive[y], list(set(lexicon_2019['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(positive[y], list(set(lexicon_2019['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2019['Tokenization'][x].split())):
                    if lexicon_2019['Tokenization'][x].split()[w] == list(set(lexicon_2019['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2019['date'][x],
                    'news_num': news_num,
                    'KOSELF_pos_word': positive[y],
                    'news_word': list(set(lexicon_2019['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(positive[y], list(set(lexicon_2019['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                pos_lexicon_2019 = pos_lexicon_2019.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2019['date'][x], positive[y], list(set(lexicon_2019['Tokenization'][x].split()))[z], ko_model.wv.similarity(positive[y], list(set(lexicon_2019['Tokenization'][x].split()))[z])))

***2019-01-02 Cosine Similarity between <경신> & <기록> : 0.522990882396698
***2019-01-02 Cosine Similarity between <중요한> & <가장> : 0.5371254086494446
***2019-01-02 Cosine Similarity between <증대> & <감소> : 0.6517643928527832
***2019-01-02 Cosine Similarity between <증대> & <증가> : 0.6946426630020142
***2019-01-02 Cosine Similarity between <해결> & <문제> : 0.6186186075210571
***2019-01-02 Cosine Similarity between <활성화> & <촉진> : 0.5009052157402039
***2019-01-02 Cosine Similarity between <본격적인> & <본격> : 0.6104379892349243
***2019-01-02 Cosine Similarity between <장점> & <특징> : 0.6442348957061768
***2019-01-02 Cosine Similarity between <추진> & <계획> : 0.5205424427986145
***2019-01-02 Cosine Similarity between <확보> & <증대> : 0.5161436796188354
***2019-01-02 Cosine Similarity between <본격적인> & <본격> : 0.6104379892349243
***2019-01-02 Cosine Similarity between <추진> & <추진> : 0.9999999403953552
***2019-01-02 Cosine Similarity between <추진> & <사업> : 0.5213737487792969
***2019-01-02 Cosine Similarity between <추진> &

In [11]:
pos_lexicon_2019

,date,news_num,KOSELF_pos_word,news_word,cosine_similarity,frequency
0,2019-01-02,1,경신,기록,0.522991,2
1,2019-01-02,1,중요한,가장,0.537125,1
2,2019-01-02,1,증대,감소,0.651764,6
3,2019-01-02,1,증대,증가,0.694643,2
4,2019-01-02,1,해결,문제,0.618619,2
...,...,...,...,...,...,...
19456,2020-01-02,3901,경신,기록,0.522991,2
19457,2020-01-02,3901,장점,단점,0.772168,1
19458,2020-01-02,3902,의심의 여지가 없는,자공시,0.518829,1
19459,2020-01-02,3902,중요한,가장,0.537125,1


In [12]:
pos_lexicon_2019.to_csv('../../../../Code/Data/Test/Stock-Year/pos_hyundai_2019.csv', index=False)

In [13]:
a = list(set(list(pos_lexicon_2019['news_word'])))

a

['창저우시',
 '잉스커츠',
 '쉐대한모터스',
 '컴퍼트델그',
 '급증',
 '인도철강공사',
 '비양쿠르',
 '실습함',
 '베뉴편',
 '실패',
 '꼽겠지',
 '컨설팅펌',
 '특징',
 '팜스코팜스코',
 '슬림하',
 '장쓰스',
 '후이저우시',
 '머락비',
 '암페어시',
 '아임벤처스',
 '맥쿼리인프라',
 '정찬',
 '나바셰바',
 '술파이드',
 '확립',
 '흐트러지',
 '텍사트인스',
 '화런윈퉁',
 '젬백스젬백스',
 '도대사',
 '방안',
 '심동희',
 '휠센터',
 '커넥티드카용',
 '칼람바',
 '폄훼하거',
 '우호',
 '플래투닝용',
 '엠에스',
 '엠아이텍',
 '폐쇄하거',
 '설진훈',
 '매직마이크',
 '박유악',
 '손넨올스네르',
 '쑹궈모터스',
 '것이므',
 '감소',
 '포뮬라원',
 '웨강다오다완취',
 '강점',
 '언택트',
 '둥펑샤오캉',
 '에어덕트',
 '그린무브',
 '횡보세',
 '문제',
 '송죽교',
 '불황',
 '소재효',
 '크래프칙',
 '임곗값',
 '마련',
 '맹주국',
 '이미드',
 '넥쏘차',
 '벳이글벳',
 '밑돈',
 '응우옌아인',
 '랙티브',
 '증가',
 '필루즈',
 '샤리오',
 '강세',
 '침구',
 '불경기',
 '프러그',
 '카투홈',
 '단점',
 '조호윤',
 '소형모듈원전',
 '카슈끄',
 '특성',
 '저하',
 '밑돌',
 '추진',
 '웨강아오다완취',
 '문제해결',
 '딥글란트',
 '프로그램매매',
 '차정훈',
 '아르티잔스',
 '천홍량',
 '하르츠개혁',
 '셀루메드셀루메드',
 '페르싱역',
 '빠링허우',
 '쟁대위',
 '운동장역',
 '와퍼주니어',
 '뉘른부르크링',
 '플래투닝할',
 '폭락',
 '확대',
 '침체',
 '랭크되',
 '어반북스',
 '원츠빌',
 '뉴파워트럭',
 '테인먼트',
 '콘렉시콘',
 '가즈프럼',
 '토픽스지수',
 '인핏손보',
 '웃돌',
 '활기'

In [14]:
b = list(set(list(pos_lexicon_2019[pos_lexicon_2019['cosine_similarity']>=0.7]['news_word'])))

b

['쎄미켐동진쎄미켐',
 '우세',
 '추진',
 '밑돈',
 '셀루메드셀루메드',
 '단점',
 '추천',
 '솔베뉴',
 '최고점',
 '실패',
 '약세',
 '불황']

In [15]:
c = list(set(list(pos_lexicon_2019[pos_lexicon_2019['cosine_similarity']>=0.65]['news_word'])))

c

['밑돈',
 '증가',
 '패너마시티비치',
 '단점',
 '실패',
 '팜스코팜스코',
 '우세',
 '추진',
 '셀루메드셀루메드',
 '추천',
 '아르티잔스',
 '솔베뉴',
 '최고점',
 '웃돌',
 '약세',
 '쎄미켐동진쎄미켐',
 '센텀광장',
 '것이므',
 '감소',
 '적극',
 '그린무브',
 '불황',
 '스테빌리티']